## Natural Language Processing
NLP is a machine learning field that understand the words in a sentence individually and the context behind using those words. Some of the common NLP tasks are - reviewing and classifying whole sentences, identifying each word as part of speech(noun, verb , adjective, entity), generating text with masked words, extracting information from the context, transalting or summarizing the text. 

### Transformers 
One of the mdoels used for solving NLP tasks is the transformers. I will be using the transformers library from Huggingface.
The pipeline API function will connect the pre-trained model with pre-processed inputs and post-prpcessing predictions to directly output the sentiment label of the sentence with accuracy score. For example- Zero-shot classification pipeline allows in annotating the text and classify the sentence into sentimental labels. Text generation pipeline will auto-complete the sentence by generatinhg the remaining text with a provided prompt. Some of the other examples include - Named entity recognition, question answering, summarization, transalation

### Categories of pre-trained transformers models
- GPT-like (auto-regressive)
- BERT-like (auto-encoding)
- BART/T5-like (sequence-to-sequence)

### Transformer architecture
-	Encoder: Encoder encode the input into numerical representation of each word.
-	Decoder: Decoder uses the encoder’s representation along with other inputs to generate a target sequence 
-	Encoder-only models: Bi-directional, for sentiment analysis (sentence classification) and named entity recognition. Ex- BERT, RoBERTa, ALBERT
-	Decoder-only models: unidirectional, for text generation, Ex – CTRL, GPT, GPT-2, Transformer XL
-	Encoder-Decoder or sequence-to-sequence models: translation or summarization, Ex – BART, mBART, Marian, T5

## Using Transformers
Stage 1 (tokenizer)  Stage 2 (Model)  Stage 3 (post processing)

Raw text -->     Input ID     -->       Logits (not probabilities)    -->    Predictions
Tokenizer: 
-	split the input into words, subwords, symbols referred as tokens
-	Mapping each token to an integer
-	“AutoTokenizer” class and “from_pretrained” method


In [3]:
import pandas as pd
# load dataset
df = pd.read_csv(r"C:\Users\minnu\Desktop\Sentiment-analysis-using-NLP\dataset_exploration\all-data.csv", names = ['label', 'sentences'], sep=',', encoding='latin-1')
df.head()

,label,sentences
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [4]:
df_2 = df.copy()

In [5]:
# Preprocessing with Tokenizer
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
raw_inputs = [df_2['sentences'][2],df_2['sentences'][4]] 
inputs = tokenizer(raw_inputs, padding= True, truncation = True, return_tensors = 'pt')
print(inputs)

{'input_ids': tensor([[  101,  1996,  2248,  4816,  3068,  2194,  3449, 19800,  4160,  2038,
          4201,  2125, 15295,  1997,  5126,  2013,  2049, 21169,  4322,  1025,
         10043,  2000,  3041,  3913, 27475,  1996,  2194, 11016,  1996,  6938,
          1997,  2049,  2436,  3667,  1010,  1996,  3679,  2695, 14428,  2229,
          2988,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  2429,  2000,  1996,  2194,  1005,  1055,  7172,  5656,  2005,
          1996,  2086,  2268,  1011,  2262,  1010, 19021,  8059,  7889,  1037,
          2146,  1011,  2744,  5658,  4341,  3930,  1999,  1996,  2846,  1997,
          2322,  1003,  1011,  2871,  1003,  2007,  2019,  4082,  5618,  7785,
          1997,  2184,  1003,  1011,  2322,  1003,  1997,  5658,  4341,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [7]:
# Going through the model
from transformers import AutoModel   # AutoModel instantiate the model from checkpoint
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)


Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# output displaying the batch size, sequence length, hidden vector dimension
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 51, 768])


In [9]:
# Model heads: making sense out of numbers
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits)

tensor([[ 1.7797, -1.6082],
        [-0.9567,  0.8728]], grad_fn=<AddmmBackward>)


In [10]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim =-1)
print(predictions)

tensor([[0.9673, 0.0327],
        [0.1383, 0.8617]], grad_fn=<SoftmaxBackward>)


## From tokenizer to model
1. Intialize the BERT model 

In [11]:
# The AutoConfig API allows to instantiate the configuration 
# of a pretrained model from any checkpoint. It contains all the information needed 
# to load the model
'''
from transformers import BertConfig, BertModel 
config = BertConfig()
model = BertModel(config)   # Model is randomly intialized
'''
from transformers import BertModel 
model = BertModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2. **Tokenizers**: Tokenizers translate raw text inputs into numerical data that can be processed by the model. It could be either word-based, character-based or subword tokenization

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = df_2["sentences"][2]
tokens = tokenizer.tokenize(sequence)

print(tokens)


['The', 'international', 'electronic', 'industry', 'company', 'El', '##cote', '##q', 'has', 'laid', 'off', 'tens', 'of', 'employees', 'from', 'its', 'Tallinn', 'facility', ';', 'contrary', 'to', 'earlier', 'lay', '##offs', 'the', 'company', 'contracted', 'the', 'ranks', 'of', 'its', 'office', 'workers', ',', 'the', 'daily', 'Post', '##ime', '##es', 'reported', '.']


In [17]:
# From tokens to input IDs
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1109, 1835, 4828, 2380, 1419, 2896, 21596, 4426, 1144, 3390, 1228, 17265, 1104, 4570, 1121, 1157, 23277, 3695, 132, 11565, 1106, 2206, 3191, 18438, 1103, 1419, 11058, 1103, 6496, 1104, 1157, 1701, 3239, 117, 1103, 3828, 3799, 10453, 1279, 2103, 119]


In [18]:
# Decode
decode_string = tokenizer.decode(ids)
print(decode_string)

The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers, the daily Postimees reported.


3. **Handling multiple sequences**: 
Model expects a batch of inputs, i.e sending multiple sentences through the model all at once. Padding makes sure all the sentences have the same length by adding a special word called padding token to the sentences with fewer values.  To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask. We can truncate the lengths of the sequences by specifying max_sequence_length parameter. 

**After understanding how tokenizer and pretrained model works on few sequences, now it's time to fine-tune the pretrained model on the whole dataset.** 

## Fine-tuning a pretrained model 
